In [2]:
import pandas as pd
import re

# =============================================================================
# RUTAS
# =============================================================================
path_data = r"C:\Users\sebib\Documents\Sebi"
path_committees = r"C:\Users\sebib\Documents\GitHub\US_Congress\data\info basica politicos"

# =============================================================================
# FUNCIONES
# =============================================================================
name_corrections = {
    'thomas macarthur': 'tom macarthur', 'gilbert cisneros': 'gil cisneros',
    'k. michael conaway': 'mike conaway', 'michael conaway': 'mike conaway',
    'thomas carper': 'tom carper', 'donald sternoff beyer': 'don beyer',
    'donald beyer': 'don beyer', 'thomas rooney': 'tom rooney',
    'robert corker': 'bob corker', 'richard allen': 'rick allen',
    'kenneth buck': 'ken buck', 'benjamin cardin': 'ben cardin',
    'robert casey': 'bob casey', 'william cassidy': 'bill cassidy',
    'earl leroy carter': 'earl carter', 'carlos mr curbelo': 'carlos curbelo',
    'charlie joseph crist': 'charlie crist', 'james hon banks': 'jim banks',
    'james banks': 'jim banks', 'david mckinley': 'david b. mckinley',
    'mark dr green': 'mark green', 'john reed': 'jack reed',
    'valerie hoyle': 'val hoyle',
}

def normalize_name(name):
    if pd.isna(name) or name == '':
        return ''
    name = str(name).strip()
    if ',' in name:
        parts = name.split(',')
        if len(parts) == 2:
            name = f"{parts[1].strip()} {parts[0].strip()}"
    for suffix in [' Jr.', ' Jr', ' Sr.', ' Sr', ' III', ' II', ' IV', ' M.D.', ' MD']:
        name = name.replace(suffix, '')
    for prefix in ['Mr. ', 'Mrs. ', 'Ms. ', 'Dr. ', 'Hon. ']:
        if name.startswith(prefix):
            name = name[len(prefix):]
    name = re.sub(r'\s+[A-Z]\.\s*', ' ', name)
    name = re.sub(r'\s+[A-Z]\s+', ' ', name)
    name = re.sub(r'\s+', ' ', name).strip()
    if name.lower() in name_corrections:
        name = name_corrections[name.lower()]
    return name

def year_to_congress(year):
    if pd.isna(year):
        return None
    year = int(year)
    if year >= 2015:
        return 114 + (year - 2015) // 2
    elif year >= 2013:
        return 113
    return None

# =============================================================================
# 1. CARGAR COMITÉS
# =============================================================================
house = pd.read_csv(f"{path_committees}/house_committee.csv", delimiter=";")
senate = pd.read_csv(f"{path_committees}/senate_committee.csv", delimiter=";")
house_119 = pd.read_csv(f"{path_committees}/house_committee_119.csv", delimiter=";")
senate_119 = pd.read_csv(f"{path_committees}/senate_committee_119.csv", delimiter=";")

house = house[house['full_name'].notna() & (house['full_name'] != '')]
senate = senate[senate['full_name'].notna() & (senate['full_name'] != '')]

for df_temp in [house, senate, house_119, senate_119]:
    if 'full_name' in df_temp.columns:
        df_temp.rename(columns={'full_name': 'name'}, inplace=True)

cols_drop = ['first_name', 'last_name', 'party', 'state', 'chamber']
for df_temp in [house, senate, house_119, senate_119]:
    df_temp.drop(columns=[c for c in cols_drop if c in df_temp.columns], inplace=True, errors='ignore')
    df_temp.drop(columns=df_temp.columns[df_temp.columns.str.contains('Unnamed')], inplace=True, errors='ignore')

committees = pd.concat([house, senate, house_119, senate_119], ignore_index=True)
committees['name_normalized'] = committees['name'].apply(normalize_name)

# =============================================================================
# 2. COMMITTEE FIELD
# =============================================================================
committee_field = {
    'Financial Services': 'Finance', 'Finance': 'Finance', 'Ways and Means': 'Finance',
    'Budget': 'Finance', 'Banking, Housing and Urban Affairs': 'Finance',
    'Armed Services': 'Defense', 'Veterans Affairs': 'Defense', 'Homeland Security': 'Defense',
    'Homeland Security and Governmental Affairs': 'Defense',
    'Energy and Commerce': 'Energy', 'Natural Resources': 'Energy',
    'Energy and Natural Resources': 'Energy', 'Environment and Public Works': 'Energy',
    'Agriculture': 'Agriculture', 'Agriculture, Nutrition and Forestry': 'Agriculture',
    'Judiciary': 'Judiciary', 'Foreign Affairs': 'Foreign Affairs', 'Foreign Relations': 'Foreign Affairs',
    'Transportation and Infrastructure': 'Infrastructure', 'Commerce, Science and Transportation': 'Infrastructure',
    'Science, Space, and Technology': 'Technology',
    'Education and Workforce': 'Health/Education', 'Health, Education, Labor and Pensions': 'Health/Education',
    'Oversight and Accountability': 'Oversight', 'House Administration': 'Oversight', 'Rules': 'Oversight',
    'Rules and Administration': 'Oversight', 'Small Business': 'Small Business',
    'Appropriations': 'Appropriations', 'Intelligence': 'Intelligence',
    'Select Committee on Intelligence': 'Intelligence',
}
committees['committee_field'] = committees['committee'].map(committee_field).fillna('Other')

# =============================================================================
# 3. CARGAR TRADES
# =============================================================================
df = pd.read_stata(f"{path_data}/congress_merged.dta")
if '_merge' in df.columns:
    df = df.drop(columns=['_merge'])
df['name_normalized'] = df['name'].apply(normalize_name)
df['trade_year'] = pd.to_datetime(df['traded'], format='%d/%m/%Y', errors='coerce').dt.year

# =============================================================================
# 4. EXPANDIR COMITÉS POR AÑO
# =============================================================================
committees_expanded = []
for _, row in committees.iterrows():
    try:
        y_start, y_end = int(row['year_start']), int(row['year_end'])
        if 2000 <= y_start <= 2030 and 2000 <= y_end <= 2030:
            for year in range(y_start, y_end + 1):
                new_row = row.copy()
                new_row['trade_year'] = year
                committees_expanded.append(new_row)
    except:
        pass
committees_expanded = pd.DataFrame(committees_expanded).drop(columns=['year_start', 'year_end'])

# =============================================================================
# 5. MODA POR CONGRESISTA
# =============================================================================
mode_committee = committees.groupby('name_normalized')['committee'].agg(lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else None).reset_index()
mode_committee.columns = ['name_normalized', 'committee_mode']
mode_role = committees.groupby('name_normalized')['role'].agg(lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else None).reset_index()
mode_role.columns = ['name_normalized', 'role_mode']
mode_field = committees.groupby('name_normalized')['committee_field'].agg(lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else None).reset_index()
mode_field.columns = ['name_normalized', 'committee_field_mode']
mode_info = mode_committee.merge(mode_role, on='name_normalized').merge(mode_field, on='name_normalized')

# =============================================================================
# 6. MERGE
# =============================================================================
merged = df.merge(committees_expanded, on=['name_normalized', 'trade_year'], how='left', indicator=True)
no_match = merged['_merge'] == 'left_only'
merged = merged.merge(mode_info, on='name_normalized', how='left')
merged.loc[no_match & merged['committee'].isna(), 'committee'] = merged.loc[no_match & merged['committee'].isna(), 'committee_mode']
merged.loc[no_match & merged['role'].isna(), 'role'] = merged.loc[no_match & merged['role'].isna(), 'role_mode']
merged.loc[no_match & merged['committee_field'].isna(), 'committee_field'] = merged.loc[no_match & merged['committee_field'].isna(), 'committee_field_mode']
merged['committee_from_mode'] = 0
merged.loc[no_match & merged['committee'].notna(), 'committee_from_mode'] = 1
merged = merged.drop(columns=['committee_mode', 'role_mode', 'committee_field_mode'])

# =============================================================================
# 7. FILTRAR Y DEDUPLICAR
# =============================================================================
merged = merged[merged['committee'].notna()].copy()

role_priority = {'Chair': 1, 'Ranking Member': 2, 'Subcommittee Chair': 3, 'Subcommittee Ranking': 4, 'Member': 5}
merged['role_priority'] = merged['role'].map(role_priority).fillna(6)
merged = merged.sort_values(['trade_id', 'role_priority']).drop_duplicates(subset='trade_id', keep='first')
merged = merged.drop(columns=['role_priority'])

# =============================================================================
# 8. UNIFICAR NOMBRES COMITÉS A INGLÉS
# =============================================================================
committee_english = {
    'Servicios armados': 'Armed Services', 'Asuntos exteriores': 'Foreign Affairs',
    'Seguridad Nacional': 'Homeland Security', 'Asignaciones': 'Appropriations',
    'Supervisión y reforma gubernamental': 'Oversight and Accountability',
    'Agricultura': 'Agriculture', 'Modos y medios': 'Ways and Means',
    'Presupuesto': 'Budget', 'Administración de la Casa': 'House Administration',
    'Transporte e Infraestructura': 'Transportation and Infrastructure',
    'Ciencia, espacio y tecnología': 'Science, Space, and Technology',
    'Asuntos de Veteranos': 'Veterans Affairs',
    'Salud, Educación, Trabajo y Pensiones': 'Health, Education, Labor and Pensions',
    'Agricultura, nutrición y silvicultura': 'Agriculture, Nutrition and Forestry',
    'Comercio, Ciencia y Transporte': 'Commerce, Science and Transportation',
    'Banca, Vivienda y Asuntos Urbanos': 'Banking, Housing and Urban Affairs',
    'Medio Ambiente y Obras Públicas': 'Environment and Public Works',
    'Energía y recursos naturales': 'Energy and Natural Resources',
    'Seguridad Nacional y Asuntos Gubernamentales': 'Homeland Security and Governmental Affairs',
    'Finanzas': 'Finance', 'Relaciones Exteriores': 'Foreign Relations',
    "Veterans' Affairs": 'Veterans Affairs',
    'Health  Education Labor and Pensions': 'Health, Education, Labor and Pensions',
    'Agriculture Nutrition and Forestry': 'Agriculture, Nutrition and Forestry',
    'Banking Housing and Urban Affairs': 'Banking, Housing and Urban Affairs',
}
merged['committee'] = merged['committee'].replace(committee_english)

# =============================================================================
# 9. COMMITTEE ID Y CHAMBER
# =============================================================================
committee_info = {
    'Foreign Affairs': {'id': 1, 'chamber': 'House'}, 'Armed Services': {'id': 2, 'chamber': 'House'},
    'Homeland Security': {'id': 3, 'chamber': 'House'}, 'Financial Services': {'id': 4, 'chamber': 'House'},
    'Energy and Commerce': {'id': 5, 'chamber': 'House'}, 'Ways and Means': {'id': 6, 'chamber': 'House'},
    'Appropriations': {'id': 7, 'chamber': 'House'}, 'Natural Resources': {'id': 8, 'chamber': 'House'},
    'Budget': {'id': 9, 'chamber': 'House'}, 'Education and Workforce': {'id': 10, 'chamber': 'House'},
    'Agriculture': {'id': 11, 'chamber': 'House'}, 'Judiciary': {'id': 12, 'chamber': 'House'},
    'Veterans Affairs': {'id': 13, 'chamber': 'House'}, 'House Administration': {'id': 14, 'chamber': 'House'},
    'Rules': {'id': 15, 'chamber': 'House'}, 'Transportation and Infrastructure': {'id': 16, 'chamber': 'House'},
    'Oversight and Accountability': {'id': 17, 'chamber': 'House'}, 'Small Business': {'id': 18, 'chamber': 'House'},
    'Intelligence': {'id': 19, 'chamber': 'House'}, 'Select Committee on Intelligence': {'id': 19, 'chamber': 'House'},
    'Select Committee on Ethics': {'id': 20, 'chamber': 'House'}, 'Science, Space, and Technology': {'id': 21, 'chamber': 'House'},
    'Finance': {'id': 101, 'chamber': 'Senate'}, 'Agriculture, Nutrition and Forestry': {'id': 102, 'chamber': 'Senate'},
    'Special Committee on Aging': {'id': 103, 'chamber': 'Senate'}, 'Committee on Indian Affairs': {'id': 104, 'chamber': 'Senate'},
    'Health, Education, Labor and Pensions': {'id': 105, 'chamber': 'Senate'},
    'Banking, Housing and Urban Affairs': {'id': 106, 'chamber': 'Senate'},
    'Homeland Security and Governmental Affairs': {'id': 107, 'chamber': 'Senate'},
    'Energy and Natural Resources': {'id': 108, 'chamber': 'Senate'}, 'Rules and Administration': {'id': 109, 'chamber': 'Senate'},
    'Commerce, Science and Transportation': {'id': 110, 'chamber': 'Senate'},
    'Environment and Public Works': {'id': 111, 'chamber': 'Senate'}, 'Foreign Relations': {'id': 112, 'chamber': 'Senate'},
}

merged['committee_id'] = merged['committee'].map(lambda x: committee_info.get(x, {}).get('id', None))
merged['committee_chamber'] = merged['chamber']

dual = {'Armed Services': {'House': 2, 'Senate': 102}, 'Appropriations': {'House': 7, 'Senate': 107},
        'Budget': {'House': 9, 'Senate': 109}, 'Agriculture': {'House': 11, 'Senate': 111},
        'Veterans Affairs': {'House': 13, 'Senate': 113}, 'Judiciary': {'House': 12, 'Senate': 114},
        'Intelligence': {'House': 19, 'Senate': 119}, 'Rules': {'House': 15, 'Senate': 115},
        'Small Business': {'House': 18, 'Senate': 118}}

def fix_id(row):
    if row['committee'] in dual:
        return dual[row['committee']].get(row['chamber'], row['committee_id'])
    return row['committee_id']

merged['committee_id'] = merged.apply(fix_id, axis=1)

# =============================================================================
# 10. CONGRESS NUMBER
# =============================================================================
merged['congress'] = merged['trade_year'].apply(year_to_congress)

# =============================================================================
# 11. LIMPIAR Y GUARDAR
# =============================================================================
merged = merged.rename(columns={'name_x': 'name'})
merged = merged.drop(columns=['name_y', '_merge'], errors='ignore')
merged['congress'] = pd.to_numeric(merged['congress'], errors='coerce')

merged.to_parquet(f"{path_data}/congress_with_committees_v2.parquet", index=False)
merged.to_stata(f"{path_data}/congress_with_committees_v2.dta", write_index=False, version=117)

print(f"✓ Guardado: {len(merged)} observaciones")
print(f"  - congress_with_committees_v2.parquet")
print(f"  - congress_with_committees_v2.dta")

✓ Guardado: 80723 observaciones
  - congress_with_committees_v2.parquet
  - congress_with_committees_v2.dta
